In [21]:
# Installation de gmpy 

!pip install gmpy2


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\cheic\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


# Importation des librairies 

In [22]:
from PIL import Image
import numpy as np
import random
import gmpy2
import math
from decimal import *

# Fonction permettant l'application de la watermark

In [23]:

def application_watermark(image,w):

	"""Cette fonction réalise l'opération de pré-markage

		Parameters
    ----------
		image : np.array 
			l'image sous forme de tableau à deux dimensions 

		w : liste de 1 et 0
			Le watermark à appliquer à l'image 

	  Returns
    -------
    np.array
        Image pré-marquée
	"""
	ligne,colone = image.shape
	for i in range(ligne):
		for j in range(colone):
			if w[j] == 1 and (image[i][j] % 2 == 0):
				image[i][j] += 1 
			if w[j] == 0 and (image[i][j] % 2 == 1):
				image[i][j] -= 1
	return(image)

# Déploiment du chiffrement de Paillier

In [24]:
""" On définit ici les paramètres nécessaires au chiffrément de Paillier 
		NB: Ces paramètres peuvent être changer.

		** Nombre de RSA **
    p : int
        un grand nombre prémier 
		q : int 
				un grand nombre prémier 

		** Clé publique **
		N : int 
			multiplication de p et q 

		** Clé privé **
		l : int
			resultat de (p-1)*(q-1) : méthode de calcul de l'indicateur d'Euler dans ce cas spécifique

		mu : int 
			inverse modulaire de la clé privée
"""
#n_length = 50
p = 976797134979797
q = 761463613663343
N = p*q
l = (p-1)*(q-1)
##g = 1+N
mu = gmpy2.invert(l,N)

"""
def L(u):
	return((u-1)//N)
"""

def chiffrement(m,r):
	"""Cette fonction réalise le chiffrement de paillier sur m 
			dans notre application m sera un bit de np.array réprésentant l'image pré-marqué 

		Parameters
    ----------
		m : int : 1 ou 0  
			bit marqué de l'image pré-marqué

		r : int
			aléa du chiffrement de paillier  

	  Returns
    -------
    int
        m chiffré 
	"""
	return( (pow(1+N,int(m),N*N) * pow(r,N,N*N))  %  (N*N))

def dechiffrement(c):
	"""Cette fonction réalise le déchiffrement de paillier sur c 
			dans notre application c sera un bit de np.array réprésentant l'image avec le message 

		Parameters
    ----------
		c : int : 1 ou 0  
			bit chiffré de l'image avec le message

	  Returns
    -------
    int
        c dechiffré 
	"""
	return ((((pow(int(c),l,N*N))-1)//N)*mu) % N 


# Fonctions permettant de chiffrer et de dechiffrer l'image


In [25]:
def chiffrement_image(img):
	"""Cette fonction réalise le chiffrement de l'image pré-marqué  

		Parameters
    ----------
		img : np.array  
			l'image pré-marqué sous forme de tableau à deux dimensions 

	  Returns
    -------
    list 
        l'image pré-marqué chiffré  sous forme d'une liste de liste 
	"""
	# Récupération des paramètres ligne et colonne de l'image pré-marqué
	ligne,colone = img.shape

	#Construction de la liste qui va recevoir l'image pré-marqué chiffré
	res = [[0 for _ in range(colone)] for _ in range(ligne)]

	#Valeur de l'aléa du chiffrément de paillier
	r = 10

	for i in range(ligne):
		for j in range(colone):
			res[i][j] = (chiffrement((img[i][j]),r))
	return res

def dechiffrement_image(img):
	"""Cette fonction réalise le déchiffrement de l'image portant le message 

		Parameters
    ----------
		img : list  
			l'image portant le message sous forme de list de list  

	  Returns
    -------
    list 
        l'image déchiffré sous forme d'une liste de liste 
	"""
	# Récupération des paramètres ligne et colonne de l'image portant le message
	img = np.array(img)
	ligne,colone = img.shape

	#Construction de la liste qui va recevoir l'image déchiffrée
	res = [[0 for _ in range(colone)] for _ in range(ligne)]
	for i in range(ligne):
		for j in range(colone):
			res[i][j] = int(dechiffrement((img[i][j])))
	return res

# Fonction d'insertion le message dans l'image

In [26]:
def insertion_message(img_crypt,message):
  """Cette fonction insert le méssage dans l'image pré-marqué et chiffré 

		Parameters
    ----------
		img_crypt : list  
			l'image pré-marqué et chiffré sous forme de list de list  

	  Returns
    -------
    np.array 
        l'image portant le méssage chiffré 
  """
  # Récupération des paramètres ligne et colonne de l'image chiffrée
  img_crypt = np.array(img_crypt)
  ligne,colone = img_crypt.shape
  r = 10
  for i in range(ligne):
    for j in range(colone):
    
      # application de la propriété d'homomorphisme additif de Paillier cryptosystème
      if message[j] == 0:
        while math.floor(Decimal(img_crypt[i][j] * chiffrement(0,r))* Decimal(4/3)) % 2 != 0:
          r = random.randint(1,N//2)
        img_crypt[i][j] = Decimal(img_crypt[i][j] * chiffrement(0,r))
      
      if message[j] == 1:
        while math.floor(Decimal(img_crypt[i][j] * chiffrement(-1,r))* Decimal(4/3)) % 2 != 1:
          r = random.randint(1,N//2)
        img_crypt[i][j] = Decimal(img_crypt[i][j] * chiffrement(-1,r))
    
  return img_crypt


# Fonction d'extraction du message directement du message chiffré

In [27]:
def extraction_msg_image_chiffre(img):
	"""Cette fonction extrait le méssage dans l'image directement sans se servir du watermark  

		Parameters
    ----------
		img : np.array  
			l'image avec le message sous forme de tableau à deux dimensions

	  Returns
    -------
    list 
        méssage inséré  
	"""

	# Récupération des paramètres ligne et colonne de l'image chiffrée
	ligne,colone = img.shape

	#list qui va récevoir le message extrait
	resultat = []

	for j in range(colone):
		if math.floor(Decimal(img[10][j])* Decimal(4/3)) % 2 == 0:
			resultat.append(0)
		else:
			resultat.append(1)
	 
	return resultat

# Fonction d'extraction du message de l'image déchiffrée

In [28]:
def extraction_msg_image_dechiffre(image,w):
	"""Cette fonction extrait le méssage dans l'image en se servant du watermark 

		Parameters
    ----------
		img : list  
			l'image avec le message sous forme de list de list

		w : liste de 1 et 0
			Le watermark à appliquer à l'image 

	  Returns
    -------
    list 
        méssage inséré  
	"""

	#Récupération des paramètres ligne et colonne de l'image chiffrée
	image = np.array(image)
	ligne,colone = image.shape

	resultat = []

	#list qui va récevoir le message extrait
	resultat2 = []

	for j in range(colone):
		if image[10][j] % 2 == 0:
			resultat.append(0)
		else:
			resultat.append(1)
	 
	for k in range(len(resultat)):
		if resultat[k] != w[k]:
			resultat2.append(1)
		else:
			resultat2.append(0)
	 
	return resultat2

# Test du programme

# Importation de l'image et transformation de l'image en noir et blanc


In [29]:
!wget https://i.ibb.co/G3MJhht/7e7.jpg

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [30]:
#Permet de garder 130 chiffres après la virgule dans les calculs effectués 
getcontext().prec = 130

# Ouverture image et convertion en noir et blanc
image = Image.open('7e7.jpg').convert('L')
image.save("gray_scale.jpg")

image = Image.open("gray_scale.jpg")

numpydata = np.array(image)
ligne,colone = numpydata.shape

#print(ligne,colone)

Watermark aléatoire d'une taille maximal de 600

In [31]:
#definition watermark
watermark = [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]
watermark = watermark[0:colone]

#Application de la watermark et sauvegarde l'image avec la watermark

In [32]:
# Appel à la fonction application_watermark
image_watermaked = application_watermark(numpydata, watermark)

# Conversion  en noir et blanc + sauvegarde 
image_watermaked = Image.fromarray(image_watermaked,mode="L")
image_watermaked.save("image_watermaked.png")

# Suite de la démo sur l'image pré-marquée

In [33]:
#Ouverture et transformation en array de l'image avec la watermark
image = Image.open('image_watermaked.png')
numpydata2 = np.array(image)
watermark = np.array(watermark)


# Ajout du message à l'image pré-marquée

In [34]:
message = [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
img_with_message = insertion_message(chiffrement_image(numpydata2),message)


# Extraction du message inséré

In [35]:
# Use case 1 : extrait le méssage dans l'image en se servant du watermark
extract = extraction_msg_image_dechiffre(dechiffrement_image(img_with_message),watermark)

# Use case 2 : extrait le méssage dans l'image sans watermark
extract2 = extraction_msg_image_chiffre(img_with_message)


# vérification

In [36]:
print("méssage extrait == méssage d'inséré : use case 1 ::")
if (extract ==  message ):
  print("OUI")
print("--------------------------------------------------------------------------------------------------------")
print("méssage extrait == méssage d'inséré : use case 2 ::")
if (extract2 ==  message ):
  print("OUI")

méssage extrait == méssage d'inséré : use case 1 ::
OUI
--------------------------------------------------------------------------------------------------------
méssage extrait == méssage d'inséré : use case 2 ::
OUI
